#### LeetCode - SQL - #602

Refer [602. Friend Requests II: Who Has the Most Friends](https://leetcode.com/problems/friend-requests-ii-who-has-the-most-friends/description/)

Write a solution to find the people who have the most friends and the most friends number.

The test cases are generated so that only one person has the most friends.

In [0]:
data_request_accepted = [[1, 2, '2016/06/03'], [1, 3, '2016/06/08'], [2, 3, '2016/06/08'], [3, 4, '2016/06/09']]
columns_request_accepted = ['requester_id', 'accepter_id', 'accept_date']
schema_request_accepted = {'requester_id':'Int64', 'accepter_id':'Int64', 'accept_date':'datetime64[ns]'}

In [0]:
import pandas

pandas_df_request_accepted = pandas.DataFrame(data=data_request_accepted, columns=columns_request_accepted).astype(schema_request_accepted)

In [0]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName("LeetCode SQL").getOrCreate()

# Spark DataFrame using Pandas DataFrame
df_request_accepted = spark_context.createDataFrame(pandas_df_request_accepted)
df_request_accepted.printSchema()
df_request_accepted.show(5)

In [0]:
df_requester = df_request_accepted.select(df_request_accepted.requester_id.alias("people_id"))
df_requester.show()

df_accepter = df_request_accepted.select(df_request_accepted.accepter_id.alias("people_id"))
df_accepter.show()

df_friends = df_requester.unionAll(df_accepter)
df_friends.show()

In [0]:
from pyspark.sql import functions as F

df_friends_grouped = df_friends.groupBy(df_friends.people_id).agg(F.count("*").alias("friend_count"))
df_friends_grouped.show()

df_friends_grouped \
    .orderBy(F.col("friend_count").desc()) \
    .offset(0).limit(1) \
    .show()